In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers

In [57]:
learning_rate = 0.01
batch_size = 64
epochs = 2
num_classes = 10

In [65]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

In [60]:
# no include FC layer
model = keras.applications.EfficientNetV2B0(include_top=True, 
                                            input_tensor=layers.Input(shape=(28, 28, 3)))
model.trainable = False

model.summary()

Model: "efficientnetv2-b0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_30 (InputLayer)          [(None, 28, 28, 3)]  0           []                               
                                                                                                  
 rescaling_21 (Rescaling)       (None, 28, 28, 3)    0           ['input_30[0][0]']               
                                                                                                  
 normalization_21 (Normalizatio  (None, 28, 28, 3)   0           ['rescaling_21[0][0]']           
 n)                                                                                               
                                                                                                  
 stem_conv (Conv2D)             (None, 14, 14, 32)   864         ['normalization_2

In [61]:
base_inputs = model.layers[0].input
# base_inputs = layers.Input(shape=(28, 28, 1))
# x = layers.UpSampling3D(size=(224, 224, 3))(base_inputs)

base_outputs = model.layers[-1].output
# base_outputs = model(x)
final_output = layers.Dense(num_classes)(base_outputs)

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

new_model = keras.Model(inputs=base_inputs, outputs=final_output)
new_model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
new_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_30 (InputLayer)          [(None, 28, 28, 3)]  0           []                               
                                                                                                  
 rescaling_21 (Rescaling)       (None, 28, 28, 3)    0           ['input_30[0][0]']               
                                                                                                  
 normalization_21 (Normalizatio  (None, 28, 28, 3)   0           ['rescaling_21[0][0]']           
 n)                                                                                               
                                                                                                  
 stem_conv (Conv2D)             (None, 14, 14, 32)   864         ['normalization_21[0][0]'] 

In [58]:
temp = tf.convert_to_tensor(x_train)
temp = tf.reshape(temp, (temp.shape[0], 28, 28, 1))
x_train = tf.image.grayscale_to_rgb(temp)

In [62]:
new_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/2


2023-01-03 00:31:07.073007: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


938/938 - 80s - loss: 2.0853 - accuracy: 0.4578 - 80s/epoch - 85ms/step
Epoch 2/2
938/938 - 75s - loss: 1.7502 - accuracy: 0.6356 - 75s/epoch - 80ms/step


In [66]:
temp = tf.convert_to_tensor(x_test)
temp = tf.reshape(temp, (temp.shape[0], 28, 28, 1))
x_test = tf.image.grayscale_to_rgb(temp)

new_model.evaluate(x_test, y_test)

2023-01-03 00:36:37.516360: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 20s 55ms/step - loss: 1.6007 - accuracy: 0.6730


[1.6007188558578491, 0.6730000376701355]